In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import pandas as pd
from scipy.special import expit
from matplotlib import pyplot as plt
import numpy as np
import bw_processing as bwp
from carculator import *

import pandas as pd


In [2]:
def replacement(cf_mass):
    return 5 - cf_mass * (1 + expit(cf_mass * 4 - 5)) + np.random.rand()

In [3]:
class MyCoolAluminiumThing:
    def __next__(self):
        cf = np.random.rand() * 2.5
        alu = replacement(cf)
        return np.array([cf, alu])

# Ecoinvent importer 

In [4]:
bd.projects.set_current("Madonna_ladra")

In [5]:
bi.databases

Databases dictionary with 0 objects

In [6]:
if 'biosphere3' in bd.databases:
    del bd.databases['biosphere3']
    
if 'ecoinvent 3.7 cutoff' in bd.databases:
    del bd.databases['ecoinvent 3.7 cutoff']

In [7]:
bi.bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/20/2021 20:01:54
  Finished: 10/20/2021 20:01:55
  Total time elapsed: 00:00:00
  CPU %: 95.90
  Memory %: 1.18
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 2.05 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



In [8]:
ei = bi.SingleOutputEcospold2Importer(r"C:/Users/lorenzou/Box Sync/PhD work/Brightway/ei/datasets/",
                                           'ecoinvent 3.7 cutoff'
                                          )
ei.apply_strategies()
ei.statistics()

Extracting XML data from 19128 datasets


KeyboardInterrupt: 

In [ ]:
ei.all_linked

In [ ]:
ei.write_database()

In [ ]:
truck = [x for x in ei if x['name'] == 'transport, freight, lorry >32 metric ton, EURO5'][0]

# Read data from carculator

In [ ]:
biosphere = bd.Database('biosphere3')
biosphere.register()

In [ ]:
ei = bd.Database('ecoinvent 3.7 cutoff')
ei.register()

In [ ]:
truck = [x for x in ei if x['name'] == 'transport, freight, lorry >32 metric ton, EURO5'][0]

In [ ]:
truck['activity']

In [ ]:
ele = ei.search('market for electricity, low-voltage RER')

In [ ]:
for act in [act for act in ei if 'market for steel,' in act['name']]:
    print(act)

In [ ]:
for act in [act for act in ei if 'aluminium' in act['name']]:
    print(act)

In [ ]:
for activity in [act for act in ei if 'Glider' in act['name']]:
    activity.delete()

    
EV = ei.new_activity(code = 'electric_vehicles', name = 'EV', unit = 'unit')
EV.save()

ICEV = ei.new_activity(code = 'electric_vehicles', name = 'EV', unit = 'unit')
ICEV.save()
    
glider = ei.new_activity(code = 'test_glider', name = 'Glider', unit = 'unit')
glider.save()

In [ ]:
steel = [act for act in ei if act['name']== 'market for steel, low-alloyed'][0]
aluminium = [act for act in ei if act['name'] == 'aluminium alloy production, AlMg3'][0]
carbon_fibre = [act for act in ei if act['name'] == 'market for carbon fibre reinforced plastic, injection moulded'][0]


In [ ]:
glider.new_exchange(input = steel.key, amount = 1300, unit = 'kg', type = 'technosphere').save()
glider.new_exchange(input = aluminium.key, amount = 200, unit = 'kg', type = 'technosphere').save()
glider.new_exchange(input = carbon_fibre.key, amount = 0, unit = 'kg', type = 'technosphere').save()

In [ ]:
for exc in glider.exchanges():
    print(exc)

In [ ]:
ICEV.new_exchange(input = glider, amount = 1, unit = 'unit').save()

EV.new_exchange(input = glider, amount = 1.1, unit = 'unit').save()

In [ ]:
for act in [act for act in ei if 'battery cell production' in act['name'] and 'RoW' in act['location']]:
    print(act)

In [ ]:
LIB = [act for act in ei if 'battery cell production' in act['name'] and 'RoW' in act['location']][0]

In [ ]:
EV.new_exchange(input = LIB, amount = 340, unit = 'kg').save()

In [ ]:
method_GWP_recipe = [m for m in bd.methods if 'ReCiPe' in str(m) and  'Midpoint (H)' in str(m) and 'climate change' in str(m)][0]
fu = {carbon_fibre:1}

In [ ]:
lca = bc.LCA(fu,method_GWP_recipe)
lca.lci()
lca.lcia()

In [ ]:
lca.score